In [13]:
# ! pip install beautifulsoup4
# ! pip install requests
# ! pip install selenium
! pip install webdriver_manager

  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata (12 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl (27 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [ ]:
import requests
from bs4 import BeautifulSoup

# URL de la page web
url = "https://tabs.ultimate-guitar.com/tab/wham/last-christmas-chords-169565"

# Envoyer une requête pour obtenir le HTML de la page
response = requests.get(url)

# Vérifier que la requête a réussi
if response.status_code == 200:
    # Parser le HTML de la page avec BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Imprimer les 500 premiers caractères du contenu HTML pour inspection
    print(soup.prettify()[:500])

    # Vérifier les classes présentes dans les divs
    for div in soup.find_all('div'):
        print(div.get('class'))
else:
    print(f"Échec de la requête, code d'état: {response.status_code}")

<!DOCTYPE html>
<html lang="en">
 <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article#">
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   LAST CHRISTMAS CHORDS by Wham! @ Ultimate-Guitar.Com
  </title>
  <meta content="To match the original recording tune a bit sharp of a half step down. [Intro] | D | % | Bm | % | Em | % | A | % | [Chorus] D Last Christmas I gave you my heart Bm But the very next day you g" name="description"/>
  <meta content="Wham! - Last Christmas (Chords), Chords, Wham!, Ultimate-Guitar.Com" name="keywords"/>
  <script>
   (function () {
        var isTouch = 'ontouchstart' in window || 'onmsgesturechange' in window;
        var isMac = ['Macintosh', 'MacIntel', 'MacPPC', 'Mac68K'].indexOf(window.navigator.platform) > -1;
        var params = new URLSearchParams(document.location.search)
        var isPrimisOnTablet = params.get('isPrimisOnTablet')

        if ((isMac && isTouch) || isPri

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

# Utiliser Chrome driver
options = webdriver.ChromeOptions()
options.add_argument('headless')  # Optionnel : pour exécuter en mode headless (sans interface graphique)
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# Ouvrir la page web
url = "https://tabs.ultimate-guitar.com/tab/rupert-holmes/escape-the-pina-colada-song-chords-512625"
driver.get(url)

# Attendre que le contenu soit chargé 
driver.implicitly_wait(10)  # Attendre jusqu'à 10 secondes que les éléments soient disponibles

# Récupérer les éléments avec la classe "P8ReX"
try:
    elements = driver.find_elements(By.CLASS_NAME, "P8ReX")
    if elements:
        for element in elements:
            text = element.text
            print(text)
    else:
        print("La section spécifiée n'a pas été trouvée.")
except Exception as e:
    print(f"Une erreur s'est produite : {e}")

# Fermer le navigateur
driver.quit()

F         Am   G    F        Dm        C        G    Dm7  C        Gm
 [Band only]
F         Am   G    F        Dm        C        Em7  F   C
 
 
[Intro]
 
      F            Am  G  F  Dm
I was tired of my lady,
          C             G  Dm7  C  Gm
We'd been together too long
        F          Am  G  F  Dm
Like a worn out recording
     C         Em7  F  C  Gm
Of a favourite song
             F         Am  G  F  Dm
So while she lay there sleepin'
           C        G  Dm7  C  Gm
I read the paper in bed
           F        Am  G  F  Dm
And in the personal columns
               C        Em7  F  C
There was this letter I read.
 
            F      Am  G  F  Dm
If you like Pina Colada
            C                G  Dm7  C Gm
And gettin' caught in the rain
         F         Am   G  F  Dm
If you're not into yoga
       C           Em7  F  C  Gm
If you have half a brain
         F              Am   G   F   Dm
If you like makin' love at midnight
       C        G    Dm7  C Gm
in the dune

In [31]:
import re

def convert_to_chordpro(input_text):
    lines = input_text.strip().split('\n')
    combined_lines = []
    buffer_lyrics = []
    buffer_chords = []

    def process_chord_line(chord_line):
        return ' '.join([f'[{chord}]' for chord in chord_line.split()])

    def process_buffers(buffer_lyrics, buffer_chords):
        if not buffer_lyrics or not buffer_chords:
            return []

        combined_lines = []
        for chords, lyrics in zip(buffer_chords, buffer_lyrics):
            processed_line = lyrics
            chord_positions = [(m.start(), m.group()) for m in re.finditer(r'[A-Ga-g][^\s]*', chords)]

            # Adjust positions for any leading spaces in the lyrics line
            adjusted_positions = []
            offset = 0
            for pos, chord in chord_positions:
                while offset < pos and pos < len(lyrics) and lyrics[pos - offset] == ' ':
                    pos += 1
                    offset += 1
                adjusted_positions.append((pos, chord))

            for pos, chord in reversed(adjusted_positions):
                processed_line = processed_line[:pos] + f'[{chord}]' + processed_line[pos:]

            combined_lines.append(processed_line)

        return combined_lines

    i = 0
    while i < len(lines):
        line = lines[i].strip()

        # Gérer les lignes d'accord seulement
        if re.match(r'^[A-Ga-g][^\s]*\s', line):
            if i + 1 < len(lines) and not re.match(r'^[A-Ga-g][^\s]*\s', lines[i + 1]):
                buffer_chords.append(line)
                buffer_lyrics.append(lines[i + 1].strip())
                i += 2
            else:
                # Traiter les lignes d'accord seulement en entourant les accords de []
                combined_lines.append(process_chord_line(line))
                i += 1

        # Gérer les balises spéciales
        elif line.startswith("[Band only]") or line.startswith("[Intro]") or line.startswith("[Hook]"):
            if buffer_lyrics or buffer_chords:
                combined_lines.extend(process_buffers(buffer_lyrics, buffer_chords))
                buffer_lyrics.clear()
                buffer_chords.clear()

            combined_lines.append(f"{{{line.strip('[]')}}}")
            i += 1

        # Gérer les sections de paroles et d'accords
        else:
            if buffer_lyrics or buffer_chords:
                combined_lines.extend(process_buffers(buffer_lyrics, buffer_chords))
                buffer_lyrics.clear()
                buffer_chords.clear()

            combined_lines.append(line)
            i += 1

    # Traitement final des buffers restants
    combined_lines.extend(process_buffers(buffer_lyrics, buffer_chords))

    return "\n".join(combined_lines)

# Exemple d'utilisation
input_text = """
F         Am   G    F        Dm        C        G    Dm7  C        Gm

[Band only]
F         Am   G    F        Dm        C        Em7  F   C

[Intro]
      F            Am  G  F  Dm
I was tired of my lady,
          C             G  Dm7  C  Gm
We'd been together too long
        F          Am  G  F  Dm
Like a worn out recording
     C         Em7  F  C  Gm
Of a favourite song
             F         Am  G  F  Dm
So while she lay there sleepin'
           C        G  Dm7  C  Gm
I read the paper in bed
           F        Am  G  F  Dm
And in the personal columns
               C        Em7  F  C
There was this letter I read.
"""

output_text = convert_to_chordpro(input_text)
print(output_text)

[F][Gm][C][Dm[Am]7][G][G][C][D[F]m]
{Band only}
[F][C][F][Em7[Am]][C][[G]Dm][F]
{Intro}
[F] [Am] [G] [F] [Dm]
[F]I was tired o[Am]f my lady,[G][F][Dm]
[C]We'd been toge[G]the[Dm7]r too lo[G[C]m]ng
[F]Like a worn out[G] recor[[Am]Dm][F]ding
[C]Of a favou[Em7]rite [F]son[C]g[Gm]
[F]So while s[Am]he l[G]ay [F]the[Dm]re sleepin'
[C]I read th[G]e p[Dm7]aper [C]in [Gm]bed
[And]C        [e]Em7  F  C[ersonal][columns]
There was this letter I read.


# on est pas loin, mais c'est pas encore ça :

In [1]:
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

def extract_and_convert(url):
    # Utiliser Chrome driver
    options = webdriver.ChromeOptions()
    options.add_argument('headless')  # Optionnel : pour exécuter en mode headless (sans interface graphique)
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

    # Ouvrir la page web
    driver.get(url)

    # Attendre que le contenu soit chargé 
    driver.implicitly_wait(10)  # Attendre jusqu'à 10 secondes que les éléments soient disponibles

    try:
        # Récupérer les éléments avec la classe "P8ReX"
        elements = driver.find_elements(By.CLASS_NAME, "P8ReX")

        if elements:
            raw_text = ''

            for element in elements:
                text = element.text
                # Remplacer les éléments entre crochets par {comment:text}
                text = re.sub(r'\[(.*?)\]', r'{comment:\1}', text)
                raw_text += text + '\n'
        else:
            driver.quit()
            raise ValueError("La section spécifiée n'a pas été trouvée.")

    except Exception as e:
        driver.quit()
        raise e

    driver.quit()

    # Convertir le texte en format ChordPro
    def convert_to_chordpro(input_text):
        lines = input_text.strip().split('\n')
        combined_lines = []
        buffer_lyrics = []
        buffer_chords = []

        def process_chord_line(chord_line):
            return ' '.join([f'[{chord}]' for chord in chord_line.split()])

        def process_buffers(buffer_lyrics, buffer_chords):
            if not buffer_lyrics or not buffer_chords:
                return []

            combined_lines = []
            for chords, lyrics in zip(buffer_chords, buffer_lyrics):
                processed_line = lyrics
                chord_positions = [(m.start(), m.group()) for m in re.finditer(r'[A-Ga-g][^\s]*', chords)]

                # Adjust positions for any leading spaces in the lyrics line
                adjusted_positions = []
                offset = 0
                for pos, chord in chord_positions:
                    while offset < pos and pos < len(lyrics) and lyrics[pos - offset] == ' ':
                        pos += 1
                        offset += 1
                    adjusted_positions.append((pos, chord))

                for pos, chord in reversed(adjusted_positions):
                    processed_line = processed_line[:pos] + f'[{chord}]' + processed_line[pos:]

                combined_lines.append(processed_line)

            return combined_lines

        i = 0
        while i < len(lines):
            line = lines[i].strip()

            # Gérer les lignes d'accord seulement
            if re.match(r'^[A-Ga-g][^\s]*\s', line):
                if i + 1 < len(lines) and not re.match(r'^[A-Ga-g][^\s]*\s', lines[i + 1]):
                    buffer_chords.append(line)
                    buffer_lyrics.append(lines[i + 1].strip())
                    i += 2
                else:
                    # Traiter les lignes d'accord seulement en entourant les accords de []
                    combined_lines.append(process_chord_line(line))
                    i += 1

            # Gérer les balises spéciales deja traitées par lignes in input
            elif line.startswith("{comment:"):
                if buffer_lyrics or buffer_chords:
                    combined_lines.extend(process_buffers(buffer_lyrics, buffer_chords))
                    buffer_lyrics.clear()
                    buffer_chords.clear()

                combined_lines.append(line)
                i += 1

            # Gérer les sections de paroles et d'accords
            else:
                if buffer_lyrics or buffer_chords:
                    combined_lines.extend(process_buffers(buffer_lyrics, buffer_chords))
                    buffer_lyrics.clear()
                    buffer_chords.clear()

                combined_lines.append(line)
                i += 1

    # Traitement final des buffers restants
        combined_lines.extend(process_buffers(buffer_lyrics, buffer_chords))

        return "\n".join(combined_lines)

    chordpro_text = convert_to_chordpro(raw_text)

    return raw_text, chordpro_text


# Exemple d'utilisation
url = "https://tabs.ultimate-guitar.com/tab/rupert-holmes/escape-the-pina-colada-song-chords-512625"
raw_text, chordpro_text = extract_and_convert(url)

# Impression des résultats
print("Texte brut:\n")
print(raw_text)
print("\nTexte ChordPro:\n")
print(chordpro_text)

Texte brut:

F         Am   G    F        Dm        C        G    Dm7  C        Gm
 {comment:Band only}
F         Am   G    F        Dm        C        Em7  F   C
 
 
{comment:Intro}
 
      F            Am  G  F  Dm
I was tired of my lady,
          C             G  Dm7  C  Gm
We'd been together too long
        F          Am  G  F  Dm
Like a worn out recording
     C         Em7  F  C  Gm
Of a favourite song
             F         Am  G  F  Dm
So while she lay there sleepin'
           C        G  Dm7  C  Gm
I read the paper in bed
           F        Am  G  F  Dm
And in the personal columns
               C        Em7  F  C
There was this letter I read.
 
            F      Am  G  F  Dm
If you like Pina Colada
            C                G  Dm7  C Gm
And gettin' caught in the rain
         F         Am   G  F  Dm
If you're not into yoga
       C           Em7  F  C  Gm
If you have half a brain
         F              Am   G   F   Dm
If you like makin' love at midnight
       C     

# une seule fonction

In [ ]:
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

def extract_and_convert(url):
    # Utiliser Chrome driver
    options = webdriver.ChromeOptions()
    options.add_argument('headless')  # Optionnel : pour exécuter en mode headless (sans interface graphique)
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

    # Ouvrir la page web
    driver.get(url)

    # Attendre que le contenu soit chargé 
    driver.implicitly_wait(10)  # Attendre jusqu'à 10 secondes que les éléments soient disponibles

    try:
        # Récupérer les éléments avec la classe "P8ReX"
        elements = driver.find_elements(By.CLASS_NAME, "P8ReX")

        if elements:
            raw_text = ''

            for element in elements:
                text = element.text
                raw_text += text + '\n'
        else:
            driver.quit()
            raise ValueError("La section spécifiée n'a pas été trouvée.")

    except Exception as e:
        driver.quit()
        raise e

    driver.quit()

    # Convertir le texte en format ChordPro
    def convert_to_chordpro(input_text):
        lines = input_text.strip().split('\n')
        combined_lines = []
        buffer_lyrics = []
        buffer_chords = []

        def process_chord_line(chord_line):
            return ' '.join([f'[{chord}]' for chord in chord_line.split()])

        def process_buffers(buffer_lyrics, buffer_chords):
            if not buffer_lyrics or not buffer_chords:
                return []

            combined_lines = []
            for chords, lyrics in zip(buffer_chords, buffer_lyrics):
                chord_positions = [(m.start(), m.group()) for m in re.finditer(r'[A-Ga-g][^\s]*', chords)]

                adjusted_positions = []
                processed_line = lyrics
                offset = 0
                for pos, chord in chord_positions:
                    while offset < pos and pos < len(lyrics) and lyrics[pos - offset] == ' ':
                        pos += 1
                        offset += 1
                    adjusted_positions.append((pos, chord))

                for pos, chord in reversed(adjusted_positions):
                    processed_line = processed_line[:pos] + f'[{chord}]' + processed_line[pos:]

                combined_lines.append(processed_line)

            return combined_lines

        i = 0
        while i < len(lines):
            line = lines[i].strip()

            # Remplacer les éléments entre crochets par {comment:text}
            if re.match(r'^\[.*?\]$', line):
                combined_lines.append(f"{{comment:{line.strip('[]')}}}")

            # Gérer les lignes d'accords seulement
            elif re.match(r'^[A-Ga-g][^\s]*\s', line):
                if i + 1 < len(lines) and not re.match(r'^[A-Ga-g][^\s]*\s', lines[i + 1]):
                    buffer_chords.append(line)
                    buffer_lyrics.append(lines[i + 1].strip())
                    i += 2
                else:
                    # Traiter les lignes d'accord seulement en entourant les accords de []
                    combined_lines.append(process_chord_line(line))
                    i += 1

            # Gérer les sections de paroles et d'accords
            else:
                if buffer_chords:
                    lyrics_line = line
                    combined_lines.extend(process_buffers([lyrics_line], buffer_chords))
                    buffer_chords.clear()
                else:
                    combined_lines.append(line)
                i += 1

        # Traitement final des buffers restants
        combined_lines.extend(process_buffers(buffer_lyrics, buffer_chords))

        return "\n".join(combined_lines)

    chordpro_text = convert_to_chordpro(raw_text)

    return raw_text, chordpro_text


# Exemple d'utilisation
url = "https://tabs.ultimate-guitar.com/tab/rupert-holmes/escape-the-pina-colada-song-chords-512625"
raw_text, chordpro_text = extract_and_convert(url)

# Impression des résultats
print("Texte brut:\n")
print(raw_text)
print("\nTexte ChordPro:\n")
print(chordpro_text)